# 0. Import packages

In [ ]:
import sys
import os

PATH_with_functions = '/home/ardhuin/TOOLS/OPTOOLS/PYTHON/'


PWD_PATH = os.getcwd()
# appending a path
sys.path.append(PATH_with_functions)
#sys.path.append('/home/mdecarlo/Documents/TOOLS/optools_FA/optools/PYTHON/codes_Marine/')


import glob
import xarray as xr
import numpy as np
import pandas as pd


from functions_cfosat_env import get_indices_macrocycles
from Misc_functions import HandlerColumnLines
from altimetry_waveforms_functions import *

# --- plotting and interactive stuff ----------------------
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as mcolors

import matplotlib.collections as mcol # to plot multilines


cNorm = mcolors.Normalize(vmin=0, vmax=2)
jet = plt.get_cmap('jet')
mpl.rcParams.update({'figure.figsize':[10,6],'axes.grid' : True,'font.size': 14,'savefig.facecolor':'white'})

PATH_save0='/home/mdecarlo/Documents/PROJETS/TCFO_Tempetes_vs_CFOSAT/WaveGroups/'


# 1. Load data

In [ ]:
islocal = 1
nbeam=2

PATH0 = '/home/datawork-cersat-public/provider/cnes/satellite/'
if islocal==1:
    PATH_L2 = '/home/mdecarlo/Documents/DATA/CFOSAT_NADIR/'
    PATH_L1A = '/home/mdecarlo/Documents/DATA/CFOSAT_L1A/'
else:
    PATH_L2 = PATH0+'l2/cfosat/swim/swi_l2____/op05/5.1.2/'
    PATH_L1A = PATH0+'l1a/cfosat/swim/swi_l1a___/op05/5.1.2/'

filel1a = PATH_L1A+'2020/045/CFO_OP05_SWI_L1A____F_20200214T090559_20200214T103900.nc'
file_L2 = PATH_L2+'2020/045/CFO_OP05_SWI_L2_____F_20200214T090559_20200214T103900.nc'

filel1a = '/home/ardhuin/PUBLI/2023_groups/DATA/CFO_OP05_SWI_L1A____F_20200214T090559_20200214T103900.nc'
file_L2 = '/home/ardhuin/PUBLI/2023_groups/DATA/CFO_OP05_SWI_L2_____F_20200214T090559_20200214T103900.nc'

dsl1a = xr.open_dataset(filel1a,decode_times=False)
DS = xr.open_dataset(file_L2,decode_times=False)


In [ ]:
dsl1a

# 1. Plots

In [ ]:
############# Retracking
Altis = np.array([519*1e3, 781*1e3, 800*1e3, 891*1e3, 1340*1e3])
namesAltis = ['CFOSAT', 'SARAL', 'ENVISAT', 'SWOT', 'Jason3']
BW = np.array([(400*1e6),(500*1e6),(320*1e6),(320*1e6),(320*1e6)])
theta3dBs=np.array([(1.6),(0.61),(1.28),(1.28),(1.28)])*np.pi/180.
ialti = 0

theta3dB=theta3dBs[ialti]
alti_sat = Altis[ialti]
bandwidth = BW[ialti]
nameSat = namesAltis[ialti]
tau=1E9/bandwidth

clight = 299792458
dr = clight * 1/(2*bandwidth)


is_echo_l1a = 1

if is_echo_l1a:
   wf_model='wf_erf2D'
   var = 'echo_l1a_0'
   max_range_fit=180
   min_range_fit=80
   nominal_tracking_gate=92
   minLS=80
else:
   wf_model='wf_brown'
   var = 'echo_l1_0'
   max_range_fit=300
   min_range_fit=205
   nominal_tracking_gate=220
   minLS=200

range_offset = nominal_tracking_gate*dr  # number of meters above nadir sea level included in waveform
    
    
issave = 0
fig,axs=plt.subplots(1,2,figsize=(14,6))
fig,axt=  plt.subplots(2,1,figsize=(18,12))
jbox=np.array([9,35])
#jbox=np.array([10,36])
for ibox in range(2):
    kbox=jbox[ibox]
# --- Select indices corresponding to macrocycles of interest ----------------
    inds = get_indices_macrocycles(DS['indices_boxes'].isel(n_box=kbox,n_posneg=1,n_beam_l1a=0))
    print('box index:',kbox)

    ds_l1 = dsl1a.isel(n_mcycles=inds,n_beam=0,n_beam0=0)
    hsat  = ds_l1['altitude']
    Re    = ds_l1['earth_radius']
    nt    = ds_l1['echo_l1_0_nt']
    flag_d= ds_l1['flag_dep_allbeams']
    misp=(ds_l1['alpha_r_p']**2+ds_l1['alpha_t_p']**2)
    Hs_native = DS['nadir_swh_native'].isel(n_mcycles=inds).values
    sig_native = 10**(DS['nadir_sigma0_native'].isel(n_mcycles=inds)/10)

# normalization only for l1a waveform
    if is_echo_l1a: 
       wfs = ds_l1[var]/sig_native
    else:
       wfs = ds_l1[var]
   
    waveforms=np.expand_dims(wfs, 0)
    misp2    =np.expand_dims(misp, 0)
    [nx,ny,nr]=np.shape(waveforms)
    ranges = np.arange(0,nr*dr,dr)    # remember to increase 35 to higher values if needed


    Hs1,ze1,Pu1,da1,R01,di1=retrack_waveforms(waveforms,ranges,max_range_fit,clight,tau=tau,\
                                    #min_method='pyramid3',wf_model=wf_model,costfun='ML',\
                                    #min_method='Nelder-Mead',wf_model=wf_model,costfun='LS',\
                                    min_method='Nelder-Mead',wf_model=wf_model,costfun='ML',\
                                    alti_sat=alti_sat,mispointing=misp2,theta3dB=theta3dB,\
                                    nominal_tracking_gate=nominal_tracking_gate,min_range_fit=min_range_fit)
    Hs2,ze2,Pu2,da2,R02,di2=retrack_waveforms(waveforms,ranges,max_range_fit,clight,tau=tau,\
                                    #min_method='pyramid3',wf_model=wf_model,costfun='LS',\
                                    min_method='Nelder-Mead',wf_model=wf_model,costfun='LS',\
                                    alti_sat=alti_sat,mispointing=misp2,theta3dB=theta3dB,\
                                    nominal_tracking_gate=nominal_tracking_gate,min_range_fit=minLS)


    print('Retracking result:',Hs1[0,ip],ze1[0,ip],di1[0,ip])
    print('Retracking result:',Hs2[0,ip],ze2[0,ip],di2[0,ip])
    print('stds  HS(native,ML,LS):',np.std(Hs_native),np.std(Hs1),np.std(Hs2))
    print('means Hs(native,ML,LS):',np.mean(Hs_native),np.mean(Hs1),np.mean(Hs2))
    print('stds   z(native,ML,LS):',None,np.std(ze1),np.std(ze2))
    print('means  z(native,ML,LS):',None,np.mean(ze1),np.mean(ze2))
    
    in1=nominal_tracking_gate-30
    in2=nominal_tracking_gate-20
    noise=np.median(np.mean(waveforms[:,:,in1:in2],axis=2))
    print('Noise level?',noise)
    ip=32
    wf_fit=wf_eval(ranges,np.array((ze1[0,ip],Hs1[0,ip],Pu1[0,ip])),clight,wf_model,noise=noise,alti_sat=alti_sat,\
                            tau=tau,nominal_tracking_gate=nominal_tracking_gate,mispointing=misp2[0,ip],theta3dB=theta3dB)
    wf_fit2=wf_eval(ranges,np.array((ze2[0,ip],Hs2[0,ip],Pu2[0,ip])),clight,wf_model,noise=noise,alti_sat=alti_sat,\
                            tau=tau,nominal_tracking_gate=nominal_tracking_gate,mispointing=misp2[0,ip],theta3dB=theta3dB)
    wf_fit3=wf_eval(ranges,np.array((0.,Hs_native[ip],Pu2[0,ip])),clight,wf_model,noise=noise,alti_sat=alti_sat,\
                            tau=tau,nominal_tracking_gate=nominal_tracking_gate,mispointing=misp2[0,ip],theta3dB=theta3dB)
    ax=axs[ibox]
#    line1=ax.plot(ranges,waveforms[0,0,:],color='k',label='waveform')
    line1=ax.plot(ranges[minLS:max_range_fit],waveforms[0,ip,minLS:max_range_fit],color='k',label='waveform')
    line3=ax.plot(ranges[minLS:max_range_fit],wf_fit2[minLS:max_range_fit],color='r',label='fitted LS')
    line2=ax.plot(ranges[min_range_fit:max_range_fit],wf_fit[min_range_fit:max_range_fit],color='b',label='fitted ML')
    line4=ax.plot(ranges[min_range_fit:max_range_fit],wf_fit3[min_range_fit:max_range_fit],color='g',label='native?')
    ax.set_xlabel('range (m)')
    ax.set_ylabel('waveform')
    #ax.set_yscale('log')
    leg = ax.legend(loc='lower right')


## ======= Box I ============================================
    ax = axt[ibox]
    line1 = ax.plot(Hs_native,color='k',linestyle='-',lw=4,zorder=20,label='native')
    line3 = ax.plot(Hs2.T,color='b',linestyle='-',lw=3,zorder=25,label='py-LS')
    line2 = ax.plot(Hs1.T,color='r',linestyle='-',lw=3,label='py-ML')
    ax.set_ylabel('Hs')
    leg = ax.legend(loc='lower right')

